In [ ]:
!pip install streamlit

In [ ]:
# iris_app.py
"""
Streamlit App for Iris Flower Classification
--------------------------------------------
This app allows:
1. Prediction Mode:
   - Users can input iris flower measurements (sliders or CSV rows).
   - App predicts species using a trained ML model.
   - Displays prediction probabilities and color-coded results.

2. Data Exploration Mode:
   - Users can explore the Iris dataset with histograms and scatter plots.
   - Toggle to color plots by species.
   - View raw dataset.

Robust model loader is included, so even if the saved model
is nested inside dicts or wrappers (GridSearchCV etc.), 
the app will still find and load it.
"""

import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
from sklearn import datasets
import matplotlib.pyplot as plt
import traceback

st.set_page_config(page_title="Iris Classifier", layout="wide")

# ---------------------------
# Robust Model Loader
# ---------------------------
# def load_model(path: str):
#     """Load model from joblib file."""
#     try:
#         model = load(path)
#         return model
#     except Exception as e:
#         st.error(f"Failed to load model: {e}")
#         st.exception(traceback.format_exc())
#         return None

def load_model(path: str):
    """Load model from joblib file and extract estimator if wrapped in dict/GridSearch."""
    try:
        obj = load(path)

        # If object itself is a model
        if hasattr(obj, "predict"):
            return obj

        # If object is a dict, look for common keys
        if isinstance(obj, dict):
            for key in ["best_estimator_", "model", "estimator", "clf"]:
                if key in obj and hasattr(obj[key], "predict"):
                    return obj[key]
            st.error("Loaded object is a dict but no valid model was found.")
            return None

        st.error("Loaded object is not a scikit-learn model.")
        return None

    except Exception as e:
        st.error(f"Failed to load model: {e}")
        st.exception(traceback.format_exc())
        return None


# ---------------------------
# Load sample data
# ---------------------------
@st.cache_resource
def load_sample_data():
    iris = datasets.load_iris(as_frame=True)
    df = iris.frame.copy()
    df = df.rename(columns={
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width',
        'target': 'target'
    })
    target_names = iris.target_names.tolist()
    df['target_name'] = df['target'].map(lambda t: target_names[int(t)])
    return df, list(df.columns[:4]), target_names

# ---------------------------
# Prediction Helper
# ---------------------------
def predict_and_format(model, X: np.ndarray, target_names):
    preds = model.predict(X)
    probs = model.predict_proba(X)
    return preds, probs, target_names

# ---------------------------
# Main App Layout
# ---------------------------
st.title("🌸 Iris Species Classifier — Predict & Explore")
st.markdown("Interactive machine learning app with prediction and data exploration modes.")

# Load model and dataset
MODEL_PATH = r"C:\Users\Admin\Downloads\iris_model.joblib"
model = load_model(MODEL_PATH)
sample_df, feature_names, target_names = load_sample_data()

# Sidebar
with st.sidebar:
    st.header("🔧 Controls")
    mode = st.radio("Select Mode", ["Prediction", "Data Exploration"], index=0)
    st.markdown("---")
    if model:
        st.success("✅ Model loaded successfully")
    else:
        st.error("❌ No model found.")
    st.info(f"Dataset: {sample_df.shape[0]} rows • Features: {', '.join(feature_names)}")

# ---------------------------
# Prediction Mode
# ---------------------------
if mode == "Prediction":
    st.subheader("🔮 Prediction Mode")
    st.markdown("Enter flower measurements below:")

    col1, col2 = st.columns([1, 1])
    with col1:
        # Sliders for features
        sepal_length = st.slider("Sepal Length (cm)", 4.0, 8.0, 5.1, step=0.1)
        sepal_width  = st.slider("Sepal Width (cm)", 2.0, 4.5, 3.5, step=0.1)
        petal_length = st.slider("Petal Length (cm)", 1.0, 7.0, 1.4, step=0.1)
        petal_width  = st.slider("Petal Width (cm)", 0.1, 2.5, 0.2, step=0.1)

        # Optional batch input
        batch_text = st.text_area(
            "Paste multiple rows (CSV, no header, format: sepal_length,sepal_width,petal_length,petal_width)",
            value="", height=80
        )
        predict_button = st.button("Predict")

    with col2:
        st.markdown("### 📊 Prediction Results")
        if predict_button:
            try:
                if batch_text.strip():
                    rows = [list(map(float, r.split(","))) for r in batch_text.strip().splitlines()]
                    X = np.array(rows)
                else:
                    X = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
                preds, probs, labels = predict_and_format(model, X, target_names)

                for i, p in enumerate(preds):
                    st.success(f"Sample {i+1}: Predicted Species = {labels[p]}")
                    st.bar_chart(pd.Series(probs[i], index=labels))

            except Exception as e:
                st.error(f"Prediction error: {e}")
                st.exception(traceback.format_exc())
        else:
            st.info("Enter values and click Predict.")

# ---------------------------
# Data Exploration Mode
# ---------------------------
else:
    st.subheader("📊 Data Exploration Mode")
    st.markdown("Explore the Iris dataset with visualizations.")

    col1, col2 = st.columns([1, 2])
    with col1:
        st.markdown("### Histogram")
        hist_feature = st.selectbox("Select feature", options=feature_names, index=0)
        bins = st.slider("Bins", 5, 50, 15)

        st.markdown("### Scatter Plot")
        x_feat = st.selectbox("X-axis", options=feature_names, index=0)
        y_feat = st.selectbox("Y-axis", options=feature_names, index=2)
        hue_by_species = st.checkbox("Color by Species", value=True)

    with col2:
        # Histogram
        fig1, ax1 = plt.subplots()
        ax1.hist(sample_df[hist_feature], bins=bins, color="skyblue", edgecolor="black")
        ax1.set_xlabel(hist_feature.title())
        ax1.set_ylabel("Count")
        st.pyplot(fig1)

        # Scatter Plot
        fig2, ax2 = plt.subplots()
        if hue_by_species:
            for sp in sample_df['target_name'].unique():
                sub = sample_df[sample_df['target_name'] == sp]
                ax2.scatter(sub[x_feat], sub[y_feat], label=sp, alpha=0.8)
            ax2.legend()
        else:
            ax2.scatter(sample_df[x_feat], sample_df[y_feat], alpha=0.8)
        ax2.set_xlabel(x_feat.title())
        ax2.set_ylabel(y_feat.title())
        st.pyplot(fig2)

    if st.checkbox("Show Raw Dataset"):
        st.dataframe(sample_df)

# ---------------------------
# Footer
# ---------------------------
st.markdown("---")
st.caption("Developed using Streamlit & Scikit-learn")


In [1]:
import subprocess

# Run Streamlit app in background
process = subprocess.Popen(
    ["streamlit", "run", "iris_app.py", "--server.port=8501", "--server.headless=true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Print the URL
print("Streamlit app is running at: http://localhost:8501/")

Streamlit app is running at: http://localhost:8501/
